In [18]:
import pandas as pd
import numpy as np
from tensorflow.python import keras
from sklearn.preprocessing import MinMaxScaler
from keras.models import  Sequential # type: ignore
from keras.layers import  Dense # type: ignore
from keras.layers import LSTM # type: ignore
#import tensorflow as tf
import matplotlib.pyplot as plt
import uuid
import re
import os

fileName = 'BEL.NS'

# Load data from Excel file
file_path = f'C:\\Raghav Proj\\StockFinder\\ZirodhaCsvFiles\\{fileName}.csv' 
folder_path = f'C:/Raghav Proj/StockFinder/ZirodhaCsvFiles' 

def genarate_png(file_path):
    pattern = r'\\([A-Z]+)\.NS\.csv'
    script = re.search(pattern, file_path).group(1)

    uid = uuid.uuid4()
    output_file_path = f'C:\\Raghav Proj\\StockFinder\\Output\\{script}_{uid}.png' 

    # Load the data
    data = pd.read_csv(file_path)
    data = data[['Date', 'Close']]  # Ensure 'Date' column is included

    # Convert 'Date' column to datetime
    data['Date'] = pd.to_datetime(data['Date'])

    # Set 'Date' as index
    data.set_index('Date', inplace=True)

    # Normalize the data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

    # Determine time_step dynamically based on data shape
    time_step = int(len(data) * 0.1)  # Example: Use 10% of data length as time_step

    # Function to create dataset with sequences
    def create_dataset(data, time_step=1):
        X, Y = [], []
        for i in range(len(data) - time_step - 1):
            X.append(data[i:(i + time_step), 0])
            Y.append(data[i + time_step, 0])
        return np.array(X), np.array(Y)

    # Prepare data with the determined time_step
    X, y = create_dataset(scaled_data, time_step)

    # Splitting data into train and test sets
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    # Reshape the data to [samples, time steps, features]
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    # Build and train the LSTM model (example architecture)
    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape=(time_step, 1)))
    model.add(LSTM(100, return_sequences=False))
    model.add(Dense(50))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')

    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=64, verbose=1, shuffle=False)

    # Predict next 5 days
    future_days = 5
    forecast = []

    # Use the last sequence from X as the initial input for prediction
    current_batch = X_test[-1].reshape((1, time_step, 1))

    for i in range(future_days):
        # Predict the next day's closing price
        current_pred = model.predict(current_batch)[0]
        
        # Append the prediction to the forecast list
        forecast.append(current_pred)
        
        # Update the current batch to include the current_pred and drop the oldest value
        current_batch = np.append(current_batch[:, 1:, :], [[current_pred]], axis=1)

    # Invert scaling to get the forecasted values in original scale
    forecast = scaler.inverse_transform(np.array(forecast).reshape(-1, 1))

    # Get dates for the forecasted period
    last_date = data.index[-1]
    forecast_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=future_days)  # Generate dates for the forecasted period

    # Concatenate original data with forecasted data for plotting
    original_dates = data.index
    plot_dates = np.concatenate((original_dates, forecast_dates))

    original_data = data['Close'].values.flatten()
    plot_data = np.concatenate((original_data, forecast.flatten()))

    # Plotting the results
    plt.figure(figsize=(10, 6))
    plt.plot(plot_dates, plot_data, label='Nifty 50 Index')
    plt.scatter(forecast_dates, forecast, color='red', label='Forecasted Data', marker='*')  # Plot forecasted data with dates
    plt.xlabel('Date')
    plt.ylabel('Closing Price')
    plt.title(script +' Nifty 50 Index Forecast for Next 5 Days')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Read all CSV files in the folder
dfs = []
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        genarate_png(file_path)

NameError: name 'os' is not defined